# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 10 2022 9:57AM,243861,19,60015311,"GUSA Granules USA, Inc.",Released
1,Aug 10 2022 9:39AM,243857,20,8292224,"ACI Healthcare USA, Inc.",Released
2,Aug 10 2022 9:27AM,243852,10,SNAP89221,"Snap Medical Industries, LLC",Released
3,Aug 10 2022 9:27AM,243852,10,SNAP89222,"Snap Medical Industries, LLC",Released
4,Aug 10 2022 9:23AM,243851,10,MSP213008,"Methapharm, Inc.",Released
5,Aug 10 2022 9:23AM,243850,12,SB-08102022-01,SugarBear,Executing
6,Aug 10 2022 9:23AM,243849,10,SOTAH0000750,"Nextsource Biotechnology, LLC",Released
7,Aug 10 2022 9:21AM,243848,10,0085930735,ISDIN Corporation,Released
8,Aug 10 2022 9:21AM,243848,10,0085930779,ISDIN Corporation,Released
9,Aug 10 2022 9:21AM,243848,10,0085930785,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,243850,Executing,1
37,243851,Released,1
38,243852,Released,2
39,243857,Released,1
40,243861,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243850,NaN,1.0,NaN
243851,NaN,NaN,1.0
243852,NaN,NaN,2.0
243857,NaN,NaN,1.0
243861,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243755,0.0,0.0,8.0
243768,21.0,4.0,0.0
243769,0.0,0.0,9.0
243771,0.0,4.0,2.0
243778,5.0,21.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243755,0,0,8
243768,21,4,0
243769,0,0,9
243771,0,4,2
243778,5,21,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243755,0,0,8
1,243768,21,4,0
2,243769,0,0,9
3,243771,0,4,2
4,243778,5,21,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243755,,,8
1,243768,21,4,
2,243769,,,9
3,243771,,4,2
4,243778,5,21,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 10 2022 9:57AM,243861,19,"GUSA Granules USA, Inc."
1,Aug 10 2022 9:39AM,243857,20,"ACI Healthcare USA, Inc."
2,Aug 10 2022 9:27AM,243852,10,"Snap Medical Industries, LLC"
4,Aug 10 2022 9:23AM,243851,10,"Methapharm, Inc."
5,Aug 10 2022 9:23AM,243850,12,SugarBear
6,Aug 10 2022 9:23AM,243849,10,"Nextsource Biotechnology, LLC"
7,Aug 10 2022 9:21AM,243848,10,ISDIN Corporation
17,Aug 10 2022 9:15AM,243846,10,Bio-PRF
25,Aug 10 2022 8:30AM,243842,16,"SHL Pharma, LLC"
26,Aug 10 2022 8:13AM,243837,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 10 2022 9:57AM,243861,19,"GUSA Granules USA, Inc.",,,1
1,Aug 10 2022 9:39AM,243857,20,"ACI Healthcare USA, Inc.",,,1
2,Aug 10 2022 9:27AM,243852,10,"Snap Medical Industries, LLC",,,2
3,Aug 10 2022 9:23AM,243851,10,"Methapharm, Inc.",,,1
4,Aug 10 2022 9:23AM,243850,12,SugarBear,,1,
5,Aug 10 2022 9:23AM,243849,10,"Nextsource Biotechnology, LLC",,,1
6,Aug 10 2022 9:21AM,243848,10,ISDIN Corporation,,,10
7,Aug 10 2022 9:15AM,243846,10,Bio-PRF,,,8
8,Aug 10 2022 8:30AM,243842,16,"SHL Pharma, LLC",,,1
9,Aug 10 2022 8:13AM,243837,10,ISDIN Corporation,,4,6


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 9:57AM,243861,19,"GUSA Granules USA, Inc.",1,,
1,Aug 10 2022 9:39AM,243857,20,"ACI Healthcare USA, Inc.",1,,
2,Aug 10 2022 9:27AM,243852,10,"Snap Medical Industries, LLC",2,,
3,Aug 10 2022 9:23AM,243851,10,"Methapharm, Inc.",1,,
4,Aug 10 2022 9:23AM,243850,12,SugarBear,,1,
5,Aug 10 2022 9:23AM,243849,10,"Nextsource Biotechnology, LLC",1,,
6,Aug 10 2022 9:21AM,243848,10,ISDIN Corporation,10,,
7,Aug 10 2022 9:15AM,243846,10,Bio-PRF,8,,
8,Aug 10 2022 8:30AM,243842,16,"SHL Pharma, LLC",1,,
9,Aug 10 2022 8:13AM,243837,10,ISDIN Corporation,6,4,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 9:57AM,243861,19,"GUSA Granules USA, Inc.",1,,
1,Aug 10 2022 9:39AM,243857,20,"ACI Healthcare USA, Inc.",1,,
2,Aug 10 2022 9:27AM,243852,10,"Snap Medical Industries, LLC",2,,
3,Aug 10 2022 9:23AM,243851,10,"Methapharm, Inc.",1,,
4,Aug 10 2022 9:23AM,243850,12,SugarBear,,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 9:57AM,243861,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
1,Aug 10 2022 9:39AM,243857,20,"ACI Healthcare USA, Inc.",1.0,NaN,NaN
2,Aug 10 2022 9:27AM,243852,10,"Snap Medical Industries, LLC",2.0,NaN,NaN
3,Aug 10 2022 9:23AM,243851,10,"Methapharm, Inc.",1.0,NaN,NaN
4,Aug 10 2022 9:23AM,243850,12,SugarBear,NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 9:57AM,243861,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
1,Aug 10 2022 9:39AM,243857,20,"ACI Healthcare USA, Inc.",1.0,0.0,0.0
2,Aug 10 2022 9:27AM,243852,10,"Snap Medical Industries, LLC",2.0,0.0,0.0
3,Aug 10 2022 9:23AM,243851,10,"Methapharm, Inc.",1.0,0.0,0.0
4,Aug 10 2022 9:23AM,243850,12,SugarBear,0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2438292,44.0,10.0,8.0
102,243818,1.0,0.0,0.0
12,487633,0.0,2.0,0.0
15,243755,8.0,0.0,0.0
16,243842,1.0,0.0,0.0
19,975257,18.0,15.0,29.0
20,1462810,55.0,66.0,5.0
21,1219140,5.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2438292,44.0,10.0,8.0
1,102,243818,1.0,0.0,0.0
2,12,487633,0.0,2.0,0.0
3,15,243755,8.0,0.0,0.0
4,16,243842,1.0,0.0,0.0
5,19,975257,18.0,15.0,29.0
6,20,1462810,55.0,66.0,5.0
7,21,1219140,5.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,44.0,10.0,8.0
1,102,1.0,0.0,0.0
2,12,0.0,2.0,0.0
3,15,8.0,0.0,0.0
4,16,1.0,0.0,0.0
5,19,18.0,15.0,29.0
6,20,55.0,66.0,5.0
7,21,5.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,44.0
1,102,Released,1.0
2,12,Released,0.0
3,15,Released,8.0
4,16,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,19,20,21
Status,,,,,,,,
Completed,8.0,0.0,0.0,0.0,0.0,29.0,5.0,0.0
Executing,10.0,0.0,2.0,0.0,0.0,15.0,66.0,0.0
Released,44.0,1.0,0.0,8.0,1.0,18.0,55.0,5.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,19,20,21
0,Completed,8.0,0.0,0.0,0.0,0.0,29.0,5.0,0.0
1,Executing,10.0,0.0,2.0,0.0,0.0,15.0,66.0,0.0
2,Released,44.0,1.0,0.0,8.0,1.0,18.0,55.0,5.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,19,20,21
0,Completed,8.0,0.0,0.0,0.0,0.0,29.0,5.0,0.0
1,Executing,10.0,0.0,2.0,0.0,0.0,15.0,66.0,0.0
2,Released,44.0,1.0,0.0,8.0,1.0,18.0,55.0,5.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()